In [1]:
import numpy as np 
import pandas as pd 
import warnings
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import roc_auc_score, accuracy_score, classification_report, confusion_matrix, roc_curve, auc
from xgboost import XGBClassifier


warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [2]:
df1 = pd.read_csv('/kaggle/input/loan-credit-risk-and-population-stability/loan_2014_18.csv')
df2 = pd.read_csv('/kaggle/input/loan-credit-risk-and-population-stability/loan_2019_20.csv')


df = pd.concat([df1, df2], ignore_index=True)


In [3]:
df.shape

(2694776, 142)

In [4]:
df.head(5)

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,pymnt_plan,url,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,fico_range_low,fico_range_high,inq_last_6mths,mths_since_last_delinq,mths_since_last_record,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,next_pymnt_d,last_credit_pull_d,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,mths_since_last_major_derog,policy_code,application_type,annual_inc_joint,dti_joint,verification_status_joint,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_bc_dlq,mths_since_recent_inq,mths_since_recent_revol_delinq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,revol_bal_joint,sec_app_fico_range_low,sec_app_fico_range_high,sec_app_earliest_cr_line,sec_app_inq_last_6mths,sec_app_mort_acc,sec_app_open_acc,sec_app_revol_util,sec_app_open_act_il,sec_app_num_rev_accts,sec_app_chargeoff_within_12_mths,sec_app_collections_12_mths_ex_med,hardship_flag,hardship_type,hardship_reason,hardship_status,deferral_term,hardship_amount,hardship_start_date,hardship_end_date,payment_plan_start_date,hardship_length,hardship_dpd,hardship_loan_status,orig_projected_additional_accrued_interest,hardship_payoff_balance_amount,hardship_last_payment_amount,debt_settlement_flag
0,0,120122535,12000.0,12000.0,12000.0,36 months,7.97%,375.88,A,A5,associate,10+ years,OWN,42000.0,Source Verified,2017-09-01,Fully Paid,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,923xx,CA,27.74,0.0,Jun-1996,715.0,719.0,0.0,NaN,80.0,9.0,1.0,11457.0,37%,16.0,w,0.00,0.00,13500.44,13500.44,12000.00,1500.44,0.0,0.0,0.0,May-2020,2591.95,NaN,May-2020,694.0,690.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,30502.0,1.0,2.0,1.0,3.0,8.0,19045.0,73.0,2.0,4.0,7117.0,53.0,31000.0,1.0,1.0,2.0,7.0,3389.0,7144.0,53.9,0.0,0.0,131.0,255.0,1.0,1.0,0.0,14.0,NaN,8.0,NaN,0.0,2.0,6.0,2.0,2.0,7.0,7.0,9.0,6.0,9.0,0.0,0.0,0.0,3.0,100.0,0.0,1.0,0.0,57180.0,30502.0,15500.0,26180.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N
1,1,119374887,32000.0,32000.0,32000.0,36 months,11.99%,1062.71,B,B5,Nurse,10+ years,MORTGAGE,155000.0,Source Verified,2017-09-01,Current,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,080xx,NJ,12.35,2.0,Sep-2005,715.0,719.0,1.0,10.0,NaN,20.0,0.0,48309.0,34.1%,42.0,w,6158.89,6158.89,31838.67,31838.67,25841.11,5997.56,0.0,0.0,0.0,May-2020,0.00,Jul-2020,May-2020,674.0,670.0,0.0,NaN,1.0,Individual,NaN,NaN,NaN,0.0,0.0,405751.0,2.0,1.0,1.0,1.0,8.0,15582.0,78.0,4.0,7.0,14049.0,40.0,142600.0,0.0,2.0,2.0,8.0,22542.0,81313.0,34.5,0.0,0.0,91.0,144.0,1.0,1.0,3.0,1.0,24.0,1.0,10.0,0.0,7.0,10.0,12.0,27.0,3.0,18.0,36.0,10.0,20.0,0.0,0.0,0.0,5.0,94.9,0.0,0.0,0.0,527034.0,63891.0,124200.0,20034.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,CVD19SKIP,INCOMECURT,ACTIVE,2.0,0.00,Apr-2020,Jun-2020,Apr-2020,2.0,0.0,ACTIVE,123.08000,6189.66,1062.71,N
2,2,119321612,40000.0,40000.0,40000.0,60 months,15.

In [5]:
df['loan_status'].unique()

array(['Fully Paid', 'Current', 'Late (31-120 days)', 'Charged Off',
       'In Grace Period', 'Late (16-30 days)', 'Default', 'Issued'],
      dtype=object)

In [6]:
# Define good vs bad loan statuses
good_statuses = ['Fully Paid', 'Current', 'Issued', 'In Grace Period']
bad_statuses = ['Charged Off', 'Default', 'Late (31-120 days)', 'Late (16-30 days)']

# Create binary target column
def map_status(status):
    if status in good_statuses:
        return 0
    elif status in bad_statuses:
        return 1

df['target'] = df['loan_status'].apply(map_status)


df.drop(['loan_status'], axis=1, inplace=True)

In [7]:
missing = (df.isna().sum() / len(df)) * 100
cols_over_40 = missing[missing > 40].index.tolist()
df = df.drop(columns=cols_over_40)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2694776 entries, 0 to 2694775
Columns: 107 entries, Unnamed: 0 to target
dtypes: float64(81), int64(3), object(23)
memory usage: 2.1+ GB


In [9]:
df.select_dtypes(include=['object']).head()

,term,int_rate,grade,sub_grade,emp_title,emp_length,home_ownership,verification_status,issue_d,pymnt_plan,url,purpose,title,zip_code,addr_state,earliest_cr_line,revol_util,initial_list_status,last_pymnt_d,last_credit_pull_d,application_type,hardship_flag,debt_settlement_flag
0,36 months,7.97%,A,A5,associate,10+ years,OWN,Source Verified,2017-09-01,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,923xx,CA,Jun-1996,37%,w,May-2020,May-2020,Individual,N,N
1,36 months,11.99%,B,B5,Nurse,10+ years,MORTGAGE,Source Verified,2017-09-01,n,https://lendingclub.com/browse/loanDetail.acti...,credit_card,Credit card refinancing,080xx,NJ,Sep-2005,34.1%,w,May-2020,May-2020,Individual,Y,N
2,60 months,15.05%,C,C4,Driver,9 years,MORTGAGE,Verified,2017-09-01,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,778xx,TX,Apr-2002,20.7%,w,May-2020,May-2020,Individual,N,N
3,36 months,7.97%,A,A5,Senior Investigator,5 years,RENT,Not Verified,2017-09-01,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,223xx,VA,Jun-2000,57.7%,w,May-2020,May-2020,Individual,N,N
4,36 months,7.21%,A,A3,Registered Nurse,< 1 year,MORTGAGE,Verified,2017-09-01,n,https://lendingclub.com/browse/loanDetail.acti...,debt_consolidation,Debt consolidation,750xx,TX,Dec-2005,16.1%,w,Apr-2020,May-2020,Joint App,Y,N


In [10]:
df.select_dtypes(include='object').nunique().sort_values(ascending=False)

url                     2694776
emp_title                465141
title                      2071
revol_util                 1344
zip_code                    959
earliest_cr_line            767
int_rate                    351
last_pymnt_d                 83
last_credit_pull_d           82
issue_d                      81
addr_state                   51
sub_grade                    35
purpose                      14
emp_length                   11
grade                         7
home_ownership                5
verification_status           3
application_type              2
hardship_flag                 2
term                          2
initial_list_status           2
debt_settlement_flag          2
pymnt_plan                    1
dtype: int64

In [11]:
df = df.apply(lambda col: col.str.strip() if col.dtype == "object" else col)

df['int_rate'] = df['int_rate'].str.replace('%', '').str.strip().astype(float)/100

df['revol_util'] = df['revol_util'].str.replace('%', '').str.strip().astype(float)/100

def clean_years_exp(x):
    if pd.isna(x):
        return x  # keep NaN as is
    x = x.strip()  # remove extra spaces
    if x == '< 1 year' or x == '< 1 years':
        return 0
    elif '10+' in x:
        return 10
    else:
        # extract the number at the start
        num = ''.join([ch for ch in x if ch.isdigit()])
        return int(num)

df['emp_length'] = df['emp_length'].apply(clean_years_exp)
df['emp_length'].unique()

array([10.,  9.,  5.,  0.,  3.,  4., nan,  6.,  2.,  8.,  7.,  1.])

In [12]:
df['hardship_flag'] = (df['hardship_flag'] == 'Y').astype(int)
df['application_type'] = (df['application_type'] == 'Joint App').astype(int)
df['term'] = (df['term'] == '60 months').astype(int)
df['initial_list_status'] = (df['initial_list_status'] == 'f').astype(int)
df['debt_settlement_flag'] = df['debt_settlement_flag'].map({'N': 0, 'Y': 1})
df['home_ownership'] = df['home_ownership'].replace(['ANY', 'NONE'], 'OTHER')

In [13]:
# Convert 'issue_d' to datetime
df['issue_d'] = pd.to_datetime(df['issue_d'], errors='coerce')

# Convert 'earliest_cr_line' to datetime
df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='%b-%Y', errors='coerce')

df['last_pymnt_d'] = pd.to_datetime(df['last_pymnt_d'], format='%b-%Y', errors='coerce')
df['last_credit_pull_d'] = pd.to_datetime(df['last_credit_pull_d'], format='%b-%Y', errors='coerce')

In [14]:
cols_to_drop = {'title','zip_code','pymnt_plan', 'emp_title', 'url', 'sub_grade'}

df.drop(cols_to_drop, axis = 1, inplace = True)

In [15]:
date_cols = df.select_dtypes(include=['datetime64']).columns
df[date_cols].isna().sum()

issue_d                  0
earliest_cr_line         0
last_pymnt_d          4688
last_credit_pull_d      61
dtype: int64

In [16]:
# 1. last_pymnt_d → replace missing with issue date
df['last_pymnt_d'] = df['last_pymnt_d'].fillna(df['issue_d'])

# 2. last_credit_pull_d → fill with max date
max_pull = df['last_credit_pull_d'].max()
df['last_credit_pull_d'] = df['last_credit_pull_d'].fillna(max_pull)

In [17]:
snapshot_date = pd.to_datetime(
    df[['issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d']].max().max()
)

In [18]:
df['loan_age_days'] = (snapshot_date - df['issue_d']).dt.days
df['credit_history_length_days'] = (df['issue_d'] - df['earliest_cr_line']).dt.days
df['days_since_last_payment'] = (snapshot_date - df['last_pymnt_d']).dt.days
df['days_since_credit_pull'] = (snapshot_date - df['last_credit_pull_d']).dt.days
df['issue_year'] = df['issue_d'].dt.year
df['issue_month'] = df['issue_d'].dt.month
df['recent_payment_flag'] = (df['days_since_last_payment'] <= 30).astype(int)

In [19]:
drop_date = {'issue_d','earliest_cr_line','last_pymnt_d','last_credit_pull_d'}
df.drop(drop_date, axis=1, inplace=True)

In [20]:
df.describe()

,Unnamed: 0,id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,emp_length,annual_inc,dti,delinq_2yrs,fico_range_low,fico_range_high,inq_last_6mths,open_acc,pub_rec,revol_bal,revol_util,total_acc,initial_list_status,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_amnt,last_fico_range_high,last_fico_range_low,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,open_acc_6m,open_act_il,open_il_12m,open_il_24m,mths_since_rcnt_il,total_bal_il,il_util,open_rv_12m,open_rv_24m,max_bal_bc,all_util,total_rev_hi_lim,inq_fi,total_cu_tl,inq_last_12m,acc_open_past_24mths,avg_cur_bal,bc_open_to_buy,bc_util,chargeoff_within_12_mths,delinq_amnt,mo_sin_old_il_acct,mo_sin_old_rev_tl_op,mo_sin_rcnt_rev_tl_op,mo_sin_rcnt_tl,mort_acc,mths_since_recent_bc,mths_since_recent_inq,num_accts_ever_120_pd,num_actv_bc_tl,num_actv_rev_tl,num_bc_sats,num_bc_tl,num_il_tl,num_op_rev_tl,num_rev_accts,num_rev_tl_bal_gt_0,num_sats,num_tl_120dpd_2m,num_tl_30dpd,num_tl_90g_dpd_24m,num_tl_op_past_12m,pct_tl_nvr_dlq,percent_bc_gt_75,pub_rec_bankruptcies,tax_liens,tot_hi_cred_lim,total_bal_ex_mort,total_bc_limit,total_il_high_credit_limit,hardship_flag,debt_settlement_flag,target,loan_age_days,credit_history_length_days,days_since_last_payment,days_since_credit_pull,issue_year,issue_month,recent_payment_flag
count,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.498555e+06,2.694776e+06,2.691668e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694775e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.692331e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2694776.0,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.059362e+06,2.059363e+06,2.059363e+06,2.059363e+06,1.998963e+06,2.059363e+06,1.768344e+06,2.059363e+06,2.059363e+06,2.059363e+06,2.059007e+06,2.694776e+06,2.059363e+06,2.059362e+06,2.059362e+06,2.694776e+06,2.694670e+06,2.663668e+06,2.662282e+06,2.694776e+06,2.694776e+06,2.615507e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.665438e+06,2.393274e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694775e+06,2.694776e+06,2.694776e+06,2.603622e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694774e+06,2.663190e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06,2.694776e+06
mean,8.751444e+04,1.058500e+08,1.549631e+04,1.549630e+04,1.549199e+04,3.006280e-01,1.297325e-01,4.550443e+02,5.823123e+00,8.064572e+04,1.954643e+01,2.953630e-01,7.005715e+02,7.045717e+02,5.284764e-01,1.176011e+01,1.830412e-01,1.705208e+04,4.832770e-01,2.401155e+01,2.208566e-01,4.384474e+03,4.383852e+03,1.232154e+04,1.231755e+04,9.720392e+03,2.430100e+03,1.820188e+00,1.692236e+02,2.972472e+01,3.444932e+03,6.909454e+02,6.785299e+02,1.897301e-02,1.0,7.811892e-02,3.315674e-03,2.252267e+02,1.458568e+05,9.312399e-01,2.826787e+00,6.988807e-01,1.603937e+00,2.040308e+01,3.683537e+04,6.896371e+01,1.253812e+00,2.645740e+00,6.018204e+03,5.627866e+01,3.672248e+04,1.071141e+00,1.517789e+00,2.001608e+00,4.528626e+00,1.370721e+04,1.297928e+04,5.518879e+01,8.324254e-03,1.016910e+01,1.259514e+02,1.797694e+02,1.448119e+01,8.309666e+00,1.492309e+00,2.503008e+01,7.115825e+00,4.974981e-01,3.704109e+00,5.603128e+00,4.867772e+00,7.537984e+00,8.579826e+00,8.279114e+00,1.371023e+01,5.555782e+00,1.172440e+01,4.912387e-04,2.182371e-03,7.766211e-02,2.088285e+00,9.423267e+01,3.916572e+01,1.265085e-01,3.825401e-02,1.842973e+05,5.331304e+04,2.506820e+04,4.608903e+04,4.510468e-02,1.826460e-02,1.285869e-01,1.265735e+03,6.009596e+03,6.715469e+02,4.099002e+02,2.016996e+03,6.487941

In [21]:
df.nunique().sort_values(ascending=False)

id                     2694776
total_pymnt            1859375
total_pymnt_inv        1467685
last_pymnt_amnt         798170
total_rec_int           696418
                        ...   
target                       2
application_type             2
term                         2
recent_payment_flag          2
policy_code                  1
Length: 104, dtype: int64

In [22]:
drop_num = {'policy_code', 'Unnamed: 0','id'}
df.drop(drop_num, axis=1, inplace=True)

In [23]:
df["fico_orig"] = (df["fico_range_low"] + df["fico_range_high"]) / 2
df["fico_recent"] = (df["last_fico_range_low"] + df["last_fico_range_high"]) / 2

df["fico_change"] = df["fico_recent"] - df["fico_orig"]

df["fico_ratio"] = df["fico_recent"] / df["fico_orig"]
df["fico_drop_flag"] = (df["fico_change"] < -20).astype(int)
df["fico_improve_flag"] = (df["fico_change"] > 20).astype(int)

fico = {'last_fico_range_high','last_fico_range_low','fico_range_low', 'fico_range_high'}
df.drop(fico , axis=1, inplace=True)

In [24]:
num_cols = df.select_dtypes(exclude=['object']).columns

df[num_cols].isna().sum()[df[num_cols].isna().sum() > 0]

emp_length               196221
dti                        3108
inq_last_6mths                1
revol_util                 2445
open_acc_6m              635414
open_act_il              635413
open_il_12m              635413
open_il_24m              635413
mths_since_rcnt_il       695813
total_bal_il             635413
il_util                  926432
open_rv_12m              635413
open_rv_24m              635413
max_bal_bc               635413
all_util                 635769
inq_fi                   635413
total_cu_tl              635414
inq_last_12m             635414
avg_cur_bal                 106
bc_open_to_buy            31108
bc_util                   32494
mo_sin_old_il_acct        79269
mths_since_recent_bc      29338
mths_since_recent_inq    301502
num_rev_accts                 1
num_tl_120dpd_2m          91154
pct_tl_nvr_dlq                2
percent_bc_gt_75          31586
dtype: int64

In [25]:
# 1. Fill with -1
fill_minus1 = [
    'mths_since_rcnt_il',
    'mo_sin_old_il_acct',
    'mths_since_recent_bc',
    'mths_since_recent_inq', 
    'open_acc_6m',
    'open_act_il',
    'open_il_12m',
    'open_il_24m',
    'open_rv_12m',
    'open_rv_24m',
    'inq_fi',
    'inq_last_12m',
    'total_cu_tl',
    'total_bal_il',
    'il_util'
]

df[fill_minus1] = df[fill_minus1].fillna(-1)

# 2. Ratio columns → median
fill_median_ratio = [
    'revol_util','il_util','all_util',
    'bc_util','percent_bc_gt_75'
]
for col in fill_median_ratio:
    df[col] = df[col].fillna(df[col].median())

# 3. Balances / dti → median
fill_median_balance = ['dti','avg_cur_bal',
                       'bc_open_to_buy', 'max_bal_bc'
]
for col in fill_median_balance:
    df[col] = df[col].fillna(df[col].median())

# 4.  counts → zero
count_cols = [
    'inq_last_6mths',
    'num_tl_120dpd_2m',
    'num_rev_accts',
    'pct_tl_nvr_dlq'
]
df[count_cols] = df[count_cols].fillna(0)

# 5. Emp length → mode
df['emp_length'] = df['emp_length'].fillna(df['emp_length'].mode()[0])

In [26]:
corr_matrix = df[num_cols].corr().abs()

# Keep only upper triangle (no self-corr or duplicate pairs)
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Filter by threshold
threshold = 0.80
high_corr = upper.stack().sort_values(ascending=False)
high_corr = high_corr[high_corr > threshold]

high_corr

loan_amnt             funded_amnt                   0.999999
out_prncp             out_prncp_inv                 0.999998
total_pymnt           total_pymnt_inv               0.999996
funded_amnt           funded_amnt_inv               0.999995
loan_amnt             funded_amnt_inv               0.999994
open_acc              num_sats                      0.999147
fico_change           fico_ratio                    0.999038
recoveries            collection_recovery_fee       0.989694
loan_age_days         issue_year                    0.986249
num_actv_rev_tl       num_rev_tl_bal_gt_0           0.985693
tot_cur_bal           tot_hi_cred_lim               0.976091
total_pymnt           total_rec_prncp               0.967086
total_pymnt_inv       total_rec_prncp               0.967079
funded_amnt           installment                   0.943125
loan_amnt             installment                   0.943124
funded_amnt_inv       installment                   0.943054
fico_recent           fi

In [27]:
to_drop = set()
target_corr = df[num_cols].corr()['target'].abs()

for (feat1, feat2), corr_val in high_corr.items():
    
    # If either feature is already dropped, skip
    if feat1 in to_drop or feat2 in to_drop:
        continue
    
    # Compare correlation with target
    corr1 = target_corr.get(feat1, 0)
    corr2 = target_corr.get(feat2, 0)
    
    # Drop the one with LOWER correlation to target
    if corr1 < corr2:
        to_drop.add(feat1)
    else:
        to_drop.add(feat2)

to_drop

{'bc_util',
 'collection_recovery_fee',
 'credit_history_length_days',
 'fico_change',
 'fico_ratio',
 'funded_amnt',
 'funded_amnt_inv',
 'issue_year',
 'loan_amnt',
 'num_actv_bc_tl',
 'num_bc_sats',
 'num_bc_tl',
 'num_op_rev_tl',
 'num_rev_accts',
 'num_rev_tl_bal_gt_0',
 'num_sats',
 'num_tl_30dpd',
 'open_acc',
 'open_il_24m',
 'open_rv_24m',
 'out_prncp_inv',
 'revol_util',
 'tot_cur_bal',
 'total_bc_limit',
 'total_il_high_credit_limit',
 'total_pymnt',
 'total_pymnt_inv'}

In [28]:
#important derived columns
df['loan_amnt_div_instlmnt']=df['loan_amnt']/df['installment']

In [29]:
df.drop(to_drop, axis=1, inplace=True)

In [30]:
df['target'].value_counts()

target
0    2348263
1     346513
Name: count, dtype: int64

In [31]:
y = df['target']

# Identify columns
cat_cols = df.select_dtypes(include=['object']).columns
num_cols = df.select_dtypes(exclude=['object']).drop('target', axis=1).columns

# OneHotEncode categorical columns
ohe = OneHotEncoder(drop='first', sparse=False, handle_unknown='ignore')
encoded = ohe.fit_transform(df[cat_cols])

# Convert encoded array to dataframe
encoded_df = pd.DataFrame(encoded, columns=ohe.get_feature_names_out(cat_cols), index=df.index)

# Combine numeric + encoded categorical
X = pd.concat([df[num_cols], encoded_df], axis=1)


In [32]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel

# 1. Fit the random forest
rf = RandomForestClassifier(
    n_estimators=150,
    random_state=42,
)
rf.fit(X_train, y_train)

# 2. Use SelectFromModel with mean importance
selector = SelectFromModel(
    rf,
    threshold="mean",  # mean importance threshold
    prefit=True        # RF is already fitted
)

# 3. Get mask of selected features
selected_mask = selector.get_support()

# 4. Get names of selected features
selected_features = X_train.columns[selected_mask].tolist()

selected_features


['term',
 'int_rate',
 'installment',
 'out_prncp',
 'total_rec_prncp',
 'total_rec_int',
 'total_rec_late_fee',
 'recoveries',
 'last_pymnt_amnt',
 'debt_settlement_flag',
 'loan_age_days',
 'days_since_last_payment',
 'days_since_credit_pull',
 'fico_recent',
 'fico_drop_flag',
 'fico_improve_flag',
 'loan_amnt_div_instlmnt']

In [34]:
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)

In [35]:
model = XGBClassifier(
    n_estimators=500,
    learning_rate=0.03,
    max_depth=6,
    subsample=0.8,
    colsample_bytree=0.8,
    scale_pos_weight=6.77,
    eval_metric="aucpr",
    tree_method="hist",
    random_state=42
)

model.fit(X_train_selected, y_train)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='aucpr', feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=6, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=500, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

In [36]:
from sklearn.metrics import average_precision_score

# ==== 1. Predictions ====
y_pred = model.predict(X_test_selected)
y_prob = model.predict_proba(X_test_selected)[:, 1]

# ==== 2. Accuracy ====
print("Accuracy:", accuracy_score(y_test, y_pred))

# ==== 3. Confusion Matrix ====
cm = confusion_matrix(y_test, y_pred)
print("\nConfusion Matrix:\n", cm)

# ==== 4. Classification Report ====
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))

# ==== 5. ROC-AUC (Recommended metric #1) ====
auc = roc_auc_score(y_test, y_prob)
print("ROC AUC:", auc)

# ==== 6. PR-AUC (Recommended metric #2 for imbalance) ====
pr_auc = average_precision_score(y_test, y_prob)
print("Precision-Recall AUC:", pr_auc)

Accuracy: 0.9951424606090293

Confusion Matrix:
 [[468350   1739]
 [   879  67988]]

Classification Report:

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    470089
           1       0.98      0.99      0.98     68867

    accuracy                           1.00    538956
   macro avg       0.99      0.99      0.99    538956
weighted avg       1.00      1.00      1.00    538956

ROC AUC: 0.9993284942319688
Precision-Recall AUC: 0.9975221762651543
